In [32]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast


model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [2]:
model_id = "gpt2"

In [22]:
model_id = "meta-llama/Meta-Llama-3-8B"

In [6]:
model_id = "facebook/opt-2.7b"

In [1]:
model_id = "EleutherAI/gpt-j-6b"

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [8]:
from linear_relational import Prompt

prompts = [
    Prompt(
        "Paris is located in the country of",
        answer="France",
        subject="Paris",
    ),
    Prompt(
        "Shanghai is located in the country of",
        answer="China",
        subject="Shanghai",
    ),
    Prompt(
        "Kyoto is located in the country of",
        answer="Japan",
        subject="Kyoto",
    ),
    Prompt(
        "San Jose is located in the country of",
        answer="Costa Rica",
        subject="San Jose",
    ),
]

In [9]:
print(model)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2560, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2560)
      (final_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-31): 32 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
            (out_proj): Linear(in_features=2560, out_features=2560, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
          (final_layer_norm): Laye

In [10]:
from linear_relational import Trainer
trainer = Trainer(model, tokenizer)

lre = trainer.train_lre(
    relation="located in country",
    subject_layer=8,
    object_layer=11,
    prompts=prompts,
)

# for prompt in prompts:
#     input_ids = tokenizer(prompt.text, return_tensors="pt")
#     logits = model(**input_ids)["logits"][0][-1]
#     print(f"{text} {tokenizer.decode(logits.argmax())} (Answer: {prompt.answer})")

validating 4 prompts


100%|████████████████████████████████████████████| 1/1 [01:40<00:00, 100.01s/it]


validating 4 prompts / 4


In [11]:
print(lre)

Lre(located in country, layers 8 -> 11, mean)


In [12]:
sample_text = "Paris is located in the country of"
sbj_pos = 0

In [29]:
# test LRE
print(f"sample_text: {sample_text}; subject position: {sbj_pos}")
input_ids = tokenizer(sample_text, return_tensors="pt")

outputs = model(**input_ids, output_hidden_states=True)
logits = outputs["logits"][0][-1]
print("From logits:", tokenizer.decode(logits.argmax()))

subject_acts = outputs.hidden_states[8][0][sbj_pos, :]  # 22-th layer, 0-th batch
object_acts_estimate = lre(subject_acts)

for i in range(12, 27):
    object_acts_estimate = model.model.decoder.layers[i](object_acts_estimate.unsqueeze(0))
object_acts_estimate = model.lm_head(object_acts_estimate)
obj_token = tokenizer.decode(object_acts_estimate.argmax())
      
print(f"From LRE   : {obj_token}")
top5 = lm_head.topk(5).indices.tolist()
print("top 5      :", [tokenizer.decode(i) for i in top5])

sample_text: Paris is located in the country of; subject position: 0
From logits:  France


ValueError: not enough values to unpack (expected 3, got 2)

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

text = "Stephen might be hiring Marcus."
emb = model.encode(text)
print(emb.shape)

(384,)
